# Pet feeder - Live Demo

The Jetson will in this demo see if the bowl is empty or full. If empty it will loose the grip (on a feeding bag), and when the bowl is full it will tighten the grip again  

### Loading the model

In [1]:
import torch
import torchvision
from torch2trt import TRTModule

device = torch.device('cuda')
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('model_trt.pth'))

<All keys matched successfully>

### Creating a preprocessing function

To match the format of the trained model to the camera, a preprocessing function is necessary. This will convert HWC layout to CHW layout, normalize, transfer data from CPU to GPU memory and add a batch dimension.

In [2]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

### Creating camera and widgets to show prediction of empty

In [3]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
empty_slider = widgets.FloatSlider(description='empty', min=0.0, max=1.0, orientation='vertical')
full_slider = widgets.FloatSlider(description='full', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

### Functions to grab or loose the grip, depending on the prediction

In [4]:
import torch.nn.functional as F
import time
from jetbot import Robot
from SCSCtrl import TTLServo

# Creating robot to enable grabber
robot = Robot()

def grab():
    TTLServo.servoAngleCtrl(4, -90, 1, 150)
    
def loose():
    TTLServo.servoAngleCtrl(4, -10, 1, 150)

def update(change):
    global empty_slider, full_slider, robot
    new_image = change['new'] 
    new_image = preprocess(new_image)
    trt_image = model_trt(new_image)
    
    # normalizing the output vector by using softmax
    trt_image = F.softmax(trt_image, dim=1)
    
    prob_empty = float(trt_image.flatten()[0])
    prob_full = float(trt_image.flatten()[1])
    
    empty_slider.value = prob_empty
    full_slider.value = prob_full
    
    # If the probability of an empty bowl is more than 80% then loose the grip
    if prob_empty > 0.8:
        loose()
    else:
        grab()
        
update({'new': camera.value})

Succeeded to open the port
Succeeded to change the baudrate


### Attaching the execution to the camera using observe 

In [5]:
camera.observe(update, names='value')

# Displaying image and sliders
display(widgets.VBox([widgets.HBox([image, empty_slider, full_slider])]))

### To stop the robot and turn off the camera

In [9]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

camera.stop()